In [ ]:
import sys
sys.path.append('D:/AI/Configuration/Scripts/AutoAugment')
# sys.path.append('../input/model-pkl/')
from autoaugment import ImageNetPolicy

import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets,  transforms, models
from collections import OrderedDict
from torch.optim.lr_scheduler import *
from collections import OrderedDict
import adabound
import numpy as np
import time


data_dir = './Cat_Dog_data'

train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       ImageNetPolicy(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])
                                                           
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True, num_workers=16, pin_memory=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64, num_workers=16, pin_memory=True)

from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b3')
model._fc = nn.Linear(1280, 2)

criterion = nn.CrossEntropyLoss()
optimizer = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)
# optimizer = optim.RMSprop(model.parameters(), lr=1e-3, momentum=0.9, eps=1e-3, weight_decay=1e-5)
scheduler = StepLR(optimizer, step_size=int(2.4 * len(trainloader)), gamma=0.97)
device = torch.device('cuda: 0' if torch.cuda.is_available() else 'cpu')
device
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

def do_deep_learning(model, trainloader, testloader, epochs, print_every, criterion, optimizer, device=device):
    epochs = epochs
    print_every = print_every
    steps = 0
    
    model.to(device)    
    
    for e in range(epochs):
        start_time = time.time()
        running_loss = 0
        for ii, (inputs, labels) in enumerate(trainloader):
            steps += 1
            scheduler.step()
            
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward and backward passes
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                print("Epoch: {}/{}... ".format(e+1, epochs),
                      "Loss: {:.4f}".format(running_loss/print_every),
                      "Time: {:.2f}s".format(time.time()-start_time))
                      
                start_time = time.time()
                running_loss = 0
        
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

    
do_deep_learning(model, trainloader, testloader, 60, 50, criterion, optimizer, device)

torch.save(model.state_dict(),'efficientnet-b5.pkl')



In [4]:
data_dir = './Cat_Dog_data'

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])
                                                           
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

testloader = torch.utils.data.DataLoader(test_data, batch_size=16, num_workers=4, pin_memory=True)
criterion = nn.CrossEntropyLoss()
running_loss = 0
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print("Loss: {:.4f}".format(running_loss/len(testloader)))
print('Accuracy of the network on the 10000 test images: %.5f %%' % (100 * correct / total))

Loss: 0.0591
Accuracy of the network on the 10000 test images: 98.68000 %


In [2]:
import torch
from torch import nn
from torchvision import datasets,  transforms
from efficientnet_pytorch import EfficientNet
device = torch.device('cuda: 0' if torch.cuda.is_available() else 'cpu')
model = EfficientNet.from_name('efficientnet-b3')
model._fc = nn.Linear(1536, 2)
model.load_state_dict(torch.load('efficientnet-b3.pkl'))
model.cuda()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        40, 24, kernel_siz

In [3]:
test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])
                                                           
test_data = datasets.ImageFolder('./data', transform=test_transforms)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, num_workers=4, pin_memory=True)

In [12]:
import numpy as np
device = torch.device('cuda: 0' if torch.cuda.is_available() else 'cpu')
test_pred = []
model.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)  
        test_pred.extend(nn.functional.softmax(outputs, dim=1)[:,1].data.cpu().numpy())

In [22]:
test_pred = np.clip(test_pred, 0.005, 0.995)

In [23]:
import pandas as pd
import os
submission_df = pd.DataFrame.from_dict({'id': os.listdir(f'./data/test'),
                                        'label': test_pred})

submission_df['id'] = submission_df['id'].map(lambda x: x.split('.')[0])
submission_df['id'] = submission_df['id'].astype(int)
submission_df = submission_df.sort_values('id')

submission_df.to_csv('submission.csv', index = False)
submission_df.head()